In [1]:
import sys
sys.path.insert(1, '/home/ubuntu/QA_code/QA_application/training/Non_Distributed')
import torch
import torch.nn as nn
import timm
import wandb
import numpy as np
import os
import random
from import_packages.dataset_partition import split_equal_into_val, split_equal_into_test # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import train_val_to_ids
from import_packages.checkpoint import save_checkpoint

In [10]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [11]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 43.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.insert(1, '/home/ubuntu/QA_code/QA_application/training/Non_Distributed')
import torch
import torch.nn as nn
import timm
import wandb
import numpy as np
import os
import random
from import_packages.dataset_partition import split_equal_into_val, split_equal_into_test # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import train_val_to_ids
from import_packages.checkpoint import save_checkpoint

In [2]:
# Deterministic Behavior
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
np.random.seed(seed)
random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [4]:
batch_size = 8
temp_train,temp_valid= split_equal_into_val(csv_file='/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_train_val_20220727-181159.csv', stratify_colname='labels',no_of_classes=2) # noqa
temp_test = split_equal_into_test(csv_file='/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_test_20220727-181159.csv', stratify_colname='labels') # noqa
partition, labels=train_val_to_ids(temp_train, temp_valid, temp_test, stratify_columns='labels') # noqa
training_set = Dataset(partition['train_set'], labels, root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/train', train_transform=True) # noqa
validation_set = Dataset(partition['val_set'],labels,root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/train',valid_transform = True) # noqa
# test_set = Dataset(partition['test_set'],labels,root_dir='/home/ubuntu/EyePacs_Lenke_Dataset_Division_64/test',test_transform=True) # noqa
train_loader = torch.utils.data.DataLoader(training_set, shuffle=True, pin_memory=True, num_workers=32, batch_size=batch_size) # noqa
val_loader = torch.utils.data.DataLoader(validation_set,shuffle=True, pin_memory=True, num_workers=32, batch_size=batch_size) # noqa

In [5]:
data_transfer = {'train': train_loader,
                 'valid': train_loader
                #  'test': test_loader
                 }

In [6]:
model_transfer = timm.create_model('convnext_tiny_in22k', pretrained=True,num_classes=2) # noqa


In [8]:
model_transfer1 = timm.create_model('resnet50d', pretrained=True,num_classes=2);model_transfer1

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [7]:
lr = 3e-4
optimizer = torch.optim.SGD(model_transfer.parameters(), lr=lr)
optimizer.zero_grad()
criterion_transfer = nn.BCEWithLogitsLoss()

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_transfer.to(device)
criterion_transfer.to(device)
# %%
if torch.cuda.device_count() > 1:
    model_transfer = nn.DataParallel(model_transfer)
    criterion_transfer = criterion_transfer
model_transfer.to(device)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate=none)
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, el

In [12]:

valid_loss_min = np.Inf
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        pred = torch.max(output,dim=1, keepdim=True)[0]
        preds = torch.max(output,dim=1, keepdim=True)[1]
        # print("Shape of output:",pred.shape)
        print("Target - ",target)
        print("Output - ",output)
        target = torch.unsqueeze(target,dim=1)
        print("Shape of output:",output.shape)
        print("Shape of output:",target.shape)
        # target = target.to(torch.double)
        # pred = pred.to(torch.double)
        loss = criterion_transfer(pred.float(), target.float())
        print("Pred", pred)
        print("Preds", preds)
        numpy_target = target.cpu().detach().numpy()
        print("Target", numpy_target)
        numpy_preds = preds.cpu().detach().numpy()
        print("Pred", numpy_preds)
        break
        

Shape of output: torch.Size([8, 2])
Shape of target: torch.Size([8])
Target -  tensor([0, 0, 0, 0, 0, 0, 0, 0])
Output -  tensor([[ 0.1953,  0.0501],
        [ 0.6919,  0.2238],
        [ 0.1733, -0.0540],
        [ 0.2371, -0.0710],
        [ 0.2819,  0.2193],
        [ 0.2986, -0.0226],
        [ 0.0736,  0.3464],
        [ 0.2294,  0.6553]], grad_fn=<AddmmBackward0>)
Shape of output: torch.Size([8, 2])
Shape of output: torch.Size([8, 1])
Pred tensor([[0.1953],
        [0.6919],
        [0.1733],
        [0.2371],
        [0.2819],
        [0.2986],
        [0.3464],
        [0.6553]], grad_fn=<MaxBackward0>)
Preds tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1]])
Target [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
Pred [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]]


In [14]:
 loss = nn.BCEWithLogitsLoss()
 input = torch.randn(3, requires_grad=True)
 target = torch.empty(3).random_(2)


In [26]:
input.shape

torch.Size([3])

In [29]:
target.shape

torch.Size([1])

In [15]:
output = loss(input, target)

In [51]:
from operator import truth


model_transfer.train()
corrects = 0.0
for batch_idx, (data, target) in enumerate(train_loader):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    output = model_transfer(data)
    #print("Output-\n", output)
    print("Target-\n",target)
    pred = torch.max(output, dim=1, keepdim=True)[0]
    preds = torch.max(output, dim=1, keepdim=True)[1]
    print("Pred-\n", pred)
    print("Preds-\n", preds)
    corrects += np.sum(preds.eq(target.data.view_as(preds)).numpy())
    print("Correct-\n",corrects)
    # print(data.size(0))
    confusion_vector = preds/target.data.view_as(preds)
    print(confusion_vector)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()
    print("true positive-",true_positives)
    print("false positive-",false_positives)
    print("true negative-",true_negatives)
    print("false negatives-",false_negatives)

    break


Target-
 tensor([0, 0, 0, 0, 0, 0, 0, 0])
Pred-
 tensor([[0.3926],
        [0.1624],
        [0.4228],
        [0.6553],
        [0.0905],
        [0.1854],
        [0.2678],
        [0.3041]], grad_fn=<MaxBackward0>)
Preds-
 tensor([[0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1]])
Correct-
 4.0
tensor([[nan],
        [nan],
        [inf],
        [inf],
        [nan],
        [inf],
        [nan],
        [inf]])
true positive- <class 'int'>
false positive- 4
true negative- 4
false negatives- 0


In [22]:
a = torch.randn(1, 3)
a

tensor([[ 0.2362,  1.1819, -0.5951]])

In [37]:
torch.max(a,keepdim=True,dim=1)

torch.return_types.max(
values=tensor([[1.1819]]),
indices=tensor([[1]]))

In [ ]:
trainAccuracy = (predicted == labels).float().sum()
trainAccuracy = 100 * trainAccuracy / labels.size(0)

In [25]:
valid_loss_min = np.Inf
model_transfer.eval()
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True,dtype), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        # # # output = torch.squeeze(output)
        # # print("Shape of output:",output.shape)
        # print("Target - ",target)
        # print("Output - ",output)
        print("Shape of target:",target.shape)
        print("Target - ",target)
        print("Output - ",output)
        pred = torch.max(output, dim=1, keepdim=True)
        loss = criterion_transfer(pred, target)
        # print(loss)
        break

SyntaxError: positional argument follows keyword argument (2377023055.py, line 12)

In [39]:

valid_loss_min = np.Inf
for epoch in range(1, 2):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0.0
    total = 0.0
    accuracy = 0.0
    model_transfer.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to('cuda', non_blocking=True,dtype), target.to('cuda', non_blocking = True) # noqa
        optimizer.zero_grad()
        output = model_transfer(data)
        print("Shape of output:",output.shape)
        print("Shape of target:",target.shape)
        # # # output = torch.squeeze(output)
        # # print("Shape of output:",output.shape)
        # print("Target - ",target)
        # print("Output - ",output)
        print("Shape of target:",target.shape)
        print("Target - ",target)
        print("Output - ",output)
        loss = criterion_transfer(pred, target)
        # print(loss)
        break

Shape of output: torch.Size([1, 2])
Shape of target: torch.Size([1])
Shape of target: torch.Size([1])
Target -  tensor([0], device='cuda:0')
Output -  tensor([[0.3863, 0.1197]], device='cuda:0', grad_fn=<GatherBackward>)


RuntimeError: result type Float can't be cast to the desired output type Long

In [4]:
import pandas as pd
df = pd.read_csv('/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_test_20220727-181159.csv')

,image,labels
0,1057328_11472605_Left Field 3.JPG,0
1,1334116_13599891_Right Field 1.JPG,0
2,926652_10417862_Left Field 3.JPG,0
3,1175273_12410031_Left Field 3.JPG,0
4,1261963_13070059_Right Field 2.JPG,0
...,...,...
512227,1314554_13455839_Right Field 3.JPG,1
512228,1132380_12071324_Right Field 1.JPG,0
512229,1120879_11978828_Right Field 3.JPG,0
512230,1325187_13535058_Right Field 3.JPG,0


In [5]:
df['labels'].value_counts()

0    485554
1     26678
Name: labels, dtype: int64

In [53]:
df1 = pd.read_csv('/home/ubuntu/QA_code/QA_application/Processed_Input_files/Split_folders/2_cases_test_20220727-181159.csv')
df1['labels'].value_counts()

0    13479
1    13479
Name: labels, dtype: int64

In [7]:
!pip install  torchmetrics

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [16]:
from torchmetrics.classification import StatScores
preds  = torch.tensor([1, 0, 2, 1])
target = torch.tensor([1, 1, 2, 0])
stat_scores = StatScores(reduce='macro', num_classes=3)
stat_scores(preds, target)

tensor([[0, 1, 2, 1, 1],
        [1, 1, 1, 1, 2],
        [1, 0, 3, 0, 1]])

In [14]:
stat_scores

StatScores()

In [17]:
stat_scores = StatScores(reduce='micro')
stat_scores(preds, target)

tensor([2, 2, 6, 2, 4])